In [29]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
%jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [30]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate
from chaogatenn.maps import LorenzMap
from chaogatenn.utils import grad_norm

In [31]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND

Y = AND_Y

In [32]:
# Map = LogisticMap(a=4.0)
Map = LorenzMap()

In [33]:
chao_gate = ChaoGate(DELTA=5.0, X0=5.0, X_THRESHOLD=5.0, Map=Map)

In [34]:
[chao_gate(x) for x in X]

[Array(5.7796865e-06, dtype=float32, weak_type=True),
 Array(3.9871746e-05, dtype=float32, weak_type=True),
 Array(3.9871746e-05, dtype=float32, weak_type=True),
 Array(0.00026231, dtype=float32, weak_type=True)]

In [35]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return -jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15))

In [36]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [37]:
optim = optax.adabelief(3e-4)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [38]:
epochs = 4_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

  0%|          | 0/4000 [00:00<?, ?it/s]

Epoch 0, Loss: 2.0615179538726807, Grad Norm: 0.34159862995147705
Epoch 10, Loss: 2.0594356060028076, Grad Norm: 0.3409695029258728
Epoch 20, Loss: 2.0565009117126465, Grad Norm: 0.3400844633579254
Epoch 30, Loss: 2.052797317504883, Grad Norm: 0.3389608860015869
Epoch 40, Loss: 2.048468828201294, Grad Norm: 0.33762702345848083
Epoch 50, Loss: 2.0435919761657715, Grad Norm: 0.33610019087791443
Epoch 60, Loss: 2.03825306892395, Grad Norm: 0.33440056443214417
Epoch 70, Loss: 2.03250789642334, Grad Norm: 0.33253538608551025
Epoch 80, Loss: 2.0264041423797607, Grad Norm: 0.3305148184299469
Epoch 90, Loss: 2.019989252090454, Grad Norm: 0.32834649085998535
Epoch 100, Loss: 2.0133004188537598, Grad Norm: 0.326043963432312
Epoch 110, Loss: 2.006371021270752, Grad Norm: 0.3236198425292969
Epoch 120, Loss: 1.999227523803711, Grad Norm: 0.3210810124874115
Epoch 130, Loss: 1.99191153049469, Grad Norm: 0.31844356656074524
Epoch 140, Loss: 1.9844435453414917, Grad Norm: 0.3157150149345398
Epoch 150, 

In [39]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: 6.986789703369141, X0: 2.1162703037261963, X_THRESHOLD: -3.660896062850952


In [40]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(chao_gate.X0 + x1 * chao_gate.DELTA + x2 * chao_gate.DELTA)
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, False),
 (False, True, False),
 (True, False, False),
 (True, True, True)]

In [41]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=1.0
